### LLM over SQL demo: LobbyIQ

Primary challenge: we need to provide more metadata and contextual information about the tables and fields to the artifical intelligence tool. In the meantime, we will be slightly more explicit about what they mean.


In [14]:
# What do you want to know?
user_question = "Consider the e_registration_linking table. Which lobbying firms have been the most active in their representation endeavours? Consider the top 10 lobbying firms, and use the most appropriate coalesced fields where available."

##### Import libraries and configure connections to DB and Azure OpenAI


In [15]:
import streamlit as st

# from openai import OpenAI
from openai import AzureOpenAI
import psycopg2
import os
from decimal import Decimal
from datetime import date
import dotenv


dotenv.load_dotenv()

# Set up OpenAI API credentials
# client = OpenAI()
client = AzureOpenAI()

# PostgreSQL connection details
DB_HOST = os.getenv("Host")
DB_PORT = os.getenv("Port")
DB_NAME = os.getenv("Database")
DB_USER = os.getenv("Username")
DB_PASSWORD = os.getenv("Password")

# Connect to PostgreSQL
conn = psycopg2.connect(
    host=DB_HOST, port=DB_PORT, dbname=DB_NAME, user=DB_USER, password=DB_PASSWORD
)


def gpt_4o_analysis(question, context):
    """
    Uses the GPT-4o model on Azure OpenAI endpoint to generate a conversational response.
    """
    messages = [
        {
            "role": "system",
            "content": "You are LobbyIQ's AI database assistant. Based on the user's question and relevant information retrieved, provide a detailed response. If there was not relevant information retrieved or you cannot answer confidently, let the user know.",
        },
        {"role": "user", "content": f"The provided context is: {context}"},
        {"role": "user", "content": f"The user's question is: {question}"},
    ]

    response = client.chat.completions.create(
        model="liq-gpt-4o",  # for Azure
        # model="gpt-4o", # for OpenAI
        # max_tokens=500,
        n=1,
        stop=None,
        temperature=0,
        messages=messages,
    )

    return response.choices[0].message.content.strip()

##### Configure LangChain SQL DB Chain


In [16]:
from langchain.chains import create_sql_query_chain
from langchain_openai import AzureChatOpenAI

pg_uri = f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri(pg_uri)
print("The detected DB dialect is " + str(db.dialect))
print("The detected DB tables are " + str(db.get_usable_table_names()))

# liq-gpt-4o; liq-gpt-35
llm = AzureChatOpenAI(model="liq-gpt-4o", temperature=0)

chain = create_sql_query_chain(llm, db)
generated_sql = chain.invoke(
    {
        "question": user_question
        + " YOU MUST RETURN ONLY PROPER, ERROR-FREE POSTGRESQL COMPLAINT QUERIES IN PLAIN-TEXT FORMAT, WITH NO ANALYSIS, WRAPPERS, OR FORMATTING SYMBOLS."
    }
)
print(
    "\n\n\n\nHere is the generated query to be executed against the database. "
    + generated_sql
)

/Users/raymondwang/.pyenv/versions/3.10.12/lib/python3.10/site-packages/langchain_community/utilities/sql_database.py:122: SAWarning: Did not recognize type 'vector' of column 'embedding'
  self._metadata.reflect(


The detected DB dialect is postgresql
The detected DB tables are ['b_commmittee_studies_mapping', 'b_studies', 'b_studies_bills_mapping', 'b_studies_briefs_mapping', 'b_studies_meetings_mapping', 'b_studies_reports_mapping', 'bill_aggregates', 'child_users', 'ci_issue_day_topics', 'ci_issue_mapping', 'ci_issues', 'ci_issues_audit', 'ci_issues_keyword_mapping', 'ci_issues_keyword_mapping_audit', 'ci_issues_tagged_topic_mapping', 'ci_issues_tagged_topic_mapping_audit', 'ci_keyword_committee_meetings', 'ci_keyword_dates', 'ci_keyword_regid', 'ci_keywords', 'ci_t_keyword_committee_meetings', 'ci_t_keyword_regid', 'ci_tagged_topics', 'ci_topic_frequency_30_day', 'clean_liq', 'combination_en_client_org_corp_nm_an', 'combined_debates', 'combined_meetings', 'committee_aggregates', 'committee_aggregates_annual', 'committee_issue_month', 'committee_issue_month_party', 'committee_issue_psm', 'committee_issue_psm_party', 'committee_mapping', 'committee_membership', 'committee_topic_psm', 'consulta

In [17]:
# Run the query
# keep only the content in generated_sql between ```sql ```
generated_sql = generated_sql.strip("```sql").strip("```")
sql_results = db.run(generated_sql)
print("The results of the SQL query are " + str(sql_results))

The results of the SQL query are [(None, 1083), ('The Capital Hill Group Inc.', 92), ('PAA Advisory | Conseils', 82), ('Crestview Strategy', 77), ('Sussex Strategy Group', 62), ('StrategyCorp Inc.', 51), ('Global Public Affairs', 50), ('Temple Scott Associates Inc.', 41), ('Earnscliffe Strategy Group', 41), ('Impact Public Affairs', 37)]


#### Analyze the SQL results


In [18]:
sql_context = f"""Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {user_question}
SQL Query: {generated_sql}
SQL Result: {sql_results}
Answer: """

# usage: gpt_4o_analysis(context, question)
final_answer = gpt_4o_analysis(sql_context, user_question)
print(final_answer)

Based on the SQL query and the result provided, the top 10 most active lobbying firms in their representation endeavors, according to the `e_registration_linking` table, are as follows:

1. **The Capital Hill Group Inc.** - 92 representations
2. **PAA Advisory | Conseils** - 82 representations
3. **Crestview Strategy** - 77 representations
4. **Sussex Strategy Group** - 62 representations
5. **StrategyCorp Inc.** - 51 representations
6. **Global Public Affairs** - 50 representations
7. **Temple Scott Associates Inc.** - 41 representations
8. **Earnscliffe Strategy Group** - 41 representations
9. **Impact Public Affairs** - 37 representations

It appears that the first entry in the SQL result is `None` with 1083 representations, which might indicate missing or uncleaned data. Therefore, it is not considered a valid lobbying firm for this context.


In [26]:
# Continue querying the database

next_query = f"""
Here is some conversation history: {final_answer}
Now, the next query is:
When were each of these lobbying firms MOST RECENTLY active? That is, what is the most recent client they took on and at what time? Firms and clients must be treated as sets (ensure the syntax passed in your query presents it as a set).
"""

from follow_up import follow_up

ans = follow_up(next_query)

/Users/raymondwang/.pyenv/versions/3.10.12/lib/python3.10/site-packages/langchain_community/utilities/sql_database.py:122: SAWarning: Did not recognize type 'vector' of column 'embedding'
  self._metadata.reflect(


The detected DB dialect is postgresql
The detected DB tables are ['b_commmittee_studies_mapping', 'b_studies', 'b_studies_bills_mapping', 'b_studies_briefs_mapping', 'b_studies_meetings_mapping', 'b_studies_reports_mapping', 'bill_aggregates', 'child_users', 'ci_issue_day_topics', 'ci_issue_mapping', 'ci_issues', 'ci_issues_audit', 'ci_issues_keyword_mapping', 'ci_issues_keyword_mapping_audit', 'ci_issues_tagged_topic_mapping', 'ci_issues_tagged_topic_mapping_audit', 'ci_keyword_committee_meetings', 'ci_keyword_dates', 'ci_keyword_regid', 'ci_keywords', 'ci_t_keyword_committee_meetings', 'ci_t_keyword_regid', 'ci_tagged_topics', 'ci_topic_frequency_30_day', 'clean_liq', 'combination_en_client_org_corp_nm_an', 'combined_debates', 'combined_meetings', 'committee_aggregates', 'committee_aggregates_annual', 'committee_issue_month', 'committee_issue_month_party', 'committee_issue_psm', 'committee_issue_psm_party', 'committee_mapping', 'committee_membership', 'committee_topic_psm', 'consulta